In [7]:
# Node and DecisionTree classes (copied from BreastCancer.ipynb for Random Forest use)
class Node():
    def __init__(self,featureIndex = None ,threshold = None,left = None ,right = None,infoGain = None ,value = None ):
        self.featureIndex = featureIndex
        self.threshold = threshold 
        self.left = left 
        self.right = right 
        self.infoGain = infoGain
        self.value = value 

import pandas as pd
import numpy as np

class DecisionTree():
    def __init__(self, minSamples, maxDepth, class_names=None):
        self.minSamples = minSamples
        self.maxDepth = maxDepth
        self.root = None
        self.class_names = class_names

    def buildTree(self, dataset, currDepth=0):
        # dataset is expected to be a pandas DataFrame where last column is the target
        X, Y = dataset.iloc[:, :-1], dataset.iloc[:, -1]
        numSamples, numCols = dataset.shape
        numFeatures = numCols - 1
        # leaf node 
        if numSamples <= self.minSamples or currDepth >= self.maxDepth:
            leafValue = self.calcLeafValue(Y)
            return Node(value=leafValue)
        bestSplit = self.getBestSplit(dataset=dataset, numSamples=numSamples, numFeatures=numFeatures)
        # if no valid split found, make a leaf
        if not bestSplit or bestSplit.get("infoGain", 0) <= 0:
            leafValue = self.calcLeafValue(Y)
            return Node(value=leafValue)
        #  build subtrees
        left_subtree = self.buildTree(bestSplit["left"], currDepth + 1)
        right_subtree = self.buildTree(bestSplit["right"], currDepth + 1)
        return Node(featureIndex=bestSplit["featureIndex"], threshold=bestSplit["threshold"], left=left_subtree, right=right_subtree, infoGain=bestSplit["infoGain"])

    def getBestSplit(self, dataset, numSamples, numFeatures):
        max_IG = -float("inf")
        bestSplit = {}
        for featureIndex in range(numFeatures):
            featureSamples = dataset.iloc[:, featureIndex]
            uniqueThresholds = np.sort(np.unique(featureSamples))
            if len(uniqueThresholds) <= 1:
                continue
            possibleThresholds = (uniqueThresholds[:-1] + uniqueThresholds[1:]) / 2.0
            for threshold in possibleThresholds:
                leftDataset, rightDataset = self.split(dataset=dataset, featureIndex=featureIndex, threshold=threshold)
                if len(leftDataset) > 0 and len(rightDataset) > 0:
                    datasetTargets = dataset.iloc[:, -1]
                    leftTargets = leftDataset.iloc[:, -1]
                    rightTargets = rightDataset.iloc[:, -1]
                    datasetEntropy = self.entropy(datasetTargets)
                    leftDatasetEntropy = self.entropy(leftTargets)
                    rightDatasetEntropy = self.entropy(rightTargets)
                    IG = self.informationGain(datasetEntropy, leftDatasetEntropy, rightDatasetEntropy, datasetTargets, leftTargets, rightTargets)
                    if IG > max_IG:
                        max_IG = IG
                        bestSplit["featureIndex"] = featureIndex
                        bestSplit["threshold"] = threshold
                        bestSplit["left"] = leftDataset
                        bestSplit["right"] = rightDataset
                        bestSplit["infoGain"] = IG
        return bestSplit

    def split(self, dataset, featureIndex, threshold):
        mask = np.array(dataset.iloc[:, featureIndex]) <= threshold
        leftDataset, rightDataset = dataset[mask], dataset[~mask]
        return leftDataset, rightDataset

    def entropy(self, dataset):
        datasetEntropy = 0.0
        numberOfExamples = len(dataset)
        classesLabels = np.unique(dataset)
        for cls in classesLabels:
            p = len(dataset[dataset == cls]) / numberOfExamples
            if p > 0:
                datasetEntropy = datasetEntropy - (p * np.log2(p))
        return datasetEntropy

    def informationGain(self, datasetEntropy, leftDatasetEntropy, rightDatasetEntropy, datasetTargets, leftTargets, rightTargets):
        l_weight = len(leftTargets) / len(datasetTargets)
        r_weight = len(rightTargets) / len(datasetTargets)
        IG = datasetEntropy - (l_weight * leftDatasetEntropy + r_weight * rightDatasetEntropy)
        return IG

    def calcLeafValue(self, Y):
        # Y may be a pandas Series; use numpy unique for counts
        classes, counts = np.unique(Y, return_counts=True)
        return classes[np.argmax(counts)]

    def fit(self, X, Y):
        # Ensure dataset is a pandas DataFrame so .iloc works in buildTree
        # X and Y may be numpy arrays; concatenate along columns and wrap in DataFrame
        dataset = pd.DataFrame(np.concatenate((X, Y), axis=1))
        self.root = self.buildTree(dataset)

    def predict(self, X):
        predictions = [self.makePrediction(x, self.root) for x in X]
        return np.array(predictions)

    def makePrediction(self, x, tree):
        if tree.value is not None:
            return tree.value
        feature = x[tree.featureIndex]
        if feature <= tree.threshold:
            return self.makePrediction(x, tree.left)
        else:
            return self.makePrediction(x, tree.right)

    def print_tree(self, tree=None, indent=" "):
        if not tree:
            tree = self.root
        if tree.value is not None:
            print(tree.value)
        else:
            print("X_" + str(tree.featureIndex), "<=", tree.threshold, "?", tree.infoGain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)


## Custom Random Forest Implementation

This section implements a Random Forest from scratch using your own DecisionTree class. Each tree is trained on a bootstrap sample, and at each split, a random subset of features is considered. The final prediction is made by majority voting across all trees.

In [10]:
# Custom Random Forest (using your DecisionTree)
import numpy as np
import random
from collections import Counter

class RandomForest:
    def __init__(self, n_trees=10, max_features=None, max_depth=None, min_samples_split=2, class_names=None, random_state=None):
        self.n_trees = n_trees
        self.max_features = max_features
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.class_names = class_names
        self.trees = []
        self.random_state = random_state
        if random_state is not None:
            np.random.seed(random_state)
            random.seed(random_state)

    def _bootstrap_sample(self, X, Y):
        n_samples = X.shape[0]
        indices = np.random.choice(n_samples, n_samples, replace=True)
        return X[indices], Y[indices]

    def _get_feature_indices(self, n_features):
        if self.max_features is None:
            return np.arange(n_features)
        else:
            return np.random.choice(n_features, self.max_features, replace=False)

    def fit(self, X, Y):
        self.trees = []
        n_features = X.shape[1]
        for i in range(self.n_trees):
            X_sample, Y_sample = self._bootstrap_sample(X, Y)
            feature_indices = self._get_feature_indices(n_features)
            tree = DecisionTree(
                minSamples=self.min_samples_split,
                maxDepth=self.max_depth,
                class_names=self.class_names
            )
            # Only use selected features for this tree
            X_tree = X_sample[:, feature_indices]
            tree.fit(X_tree, Y_sample)
            self.trees.append((tree, feature_indices))

    def predict(self, X):
        # Each tree predicts using its own feature subset
        tree_preds = []
        for tree, feature_indices in self.trees:
            X_sub = X[:, feature_indices]
            preds = tree.predict(X_sub)
            tree_preds.append(preds)
        # Majority vote
        tree_preds = np.array(tree_preds)  # shape: (n_trees, n_samples)
        y_pred = []
        for i in range(X.shape[0]):
            votes = tree_preds[:, i]
            # If class_names is set, map back to numeric for voting
            if self.class_names is not None and isinstance(self.class_names, (list, tuple)):
                # Map class names to indices for voting
                votes = [self.class_names.index(v) if v in self.class_names else v for v in votes]
            vote = Counter(votes).most_common(1)[0][0]
            # If class_names, map back to name
            if self.class_names is not None and isinstance(self.class_names, (list, tuple)):
                vote = self.class_names[int(vote)]
            y_pred.append(vote)
        return np.array(y_pred)


In [11]:
# D3. Hyperparameter Tuning and Evaluation for Custom Random Forest
from math import sqrt, floor
import pandas as pd
from sklearn.metrics import accuracy_score
# Load data (Breast Cancer Wisconsin)
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
X = data.data
Y = data.target.reshape(-1, 1)
feature_names = data.feature_names
class_names = list(data.target_names)

# 70/15/15 split (train/val/test)
from sklearn.model_selection import train_test_split
X_trainval, X_test, Y_trainval, Y_test = train_test_split(X, Y, test_size=0.15, random_state=42, stratify=Y)
X_train, X_val, Y_train, Y_val = train_test_split(X_trainval, Y_trainval, test_size=0.1765, random_state=42, stratify=Y_trainval)
# 0.1765 ≈ 15/(85) to get 70/15/15

print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

# Use best max_depth and min_samples_split from Part B (replace with your actual values)
best_max_depth = 4  # <-- set this to your best value from Part B
best_min_samples = 2  # <-- set this to your best value from Part B

d = X.shape[1]
grid_T = [5, 10, 30, 50]
grid_max_features = [floor(sqrt(d)), floor(d/2)]

results = []
for T in grid_T:
    for max_feat in grid_max_features:
        rf = RandomForest(
            n_trees=T,
            max_features=max_feat,
            max_depth=best_max_depth,
            min_samples_split=best_min_samples,
            class_names=class_names,
            random_state=42
        )
        rf.fit(X_train,Y_train)
        Y_val_pred = rf.predict(X_val)
        acc = accuracy_score(Y_val, Y_val_pred)
        results.append({
            'T': T,
            'max_features': max_feat,
            'val_accuracy': acc
        })
        print(f"T={T}, max_features={max_feat}: Val Accuracy={acc:.4f}")

# Select best hyperparameters
results_df = pd.DataFrame(results)
best_idx = results_df['val_accuracy'].idxmax()
best_params = results_df.iloc[best_idx]
print("\nBest hyperparameters:")
print(best_params)

# Retrain on train+val with best params, evaluate on test set
rf_final = RandomForest(
    n_trees=int(best_params['T']),
    max_features=int(best_params['max_features']),
    max_depth=best_max_depth,
    min_samples_split=best_min_samples,
    class_names=class_names,
    random_state=42
)
rf_final.fit(np.vstack([X_train, X_val]), np.vstack([Y_train, Y_val]))
Y_test_pred = rf_final.predict(X_test)
test_acc = accuracy_score(Y_test, Y_test_pred)
print(f"\nTest Accuracy (Random Forest): {test_acc:.4f}")

Train: (397, 30), Val: (86, 30), Test: (86, 30)
T=5, max_features=5: Val Accuracy=0.0000
T=5, max_features=5: Val Accuracy=0.0000
T=5, max_features=15: Val Accuracy=0.0000
T=5, max_features=15: Val Accuracy=0.0000
T=10, max_features=5: Val Accuracy=0.0000
T=10, max_features=5: Val Accuracy=0.0000
T=10, max_features=15: Val Accuracy=0.0000
T=10, max_features=15: Val Accuracy=0.0000
T=30, max_features=5: Val Accuracy=0.0000
T=30, max_features=5: Val Accuracy=0.0000
T=30, max_features=15: Val Accuracy=0.0000
T=30, max_features=15: Val Accuracy=0.0000


KeyboardInterrupt: 

## Random Forest Performance Analysis

Evaluate the trained Random Forest on the test set: accuracy, precision, recall, F1-score, confusion matrix, and feature importance. Compare with the single Decision Tree results.

In [ ]:
# Performance Metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

print("=" * 60)
print("RANDOM FOREST PERFORMANCE ON TEST SET")
print("=" * 60)

# Compute metrics
accuracy = accuracy_score(Y_test, Y_test_pred)
precision = precision_score(Y_test, Y_test_pred, average=None)
recall = recall_score(Y_test, Y_test_pred, average=None)
f1 = f1_score(Y_test, Y_test_pred, average=None)

print(f"\nOverall Accuracy: {accuracy:.4f}")
print("\nPer-Class Metrics:")
for i, class_name in enumerate(class_names):
    print(f"\n{class_name.upper()}:")
    print(f"  - Precision: {precision[i]:.4f}")
    print(f"  - Recall: {recall[i]:.4f}")
    print(f"  - F1-Score: {f1[i]:.4f}")

print("\n" + "=" * 60)
print("Classification Report")
print("=" * 60)
print(classification_report(Y_test, Y_test_pred, target_names=class_names))

In [ ]:
# Confusion Matrix Visualization
import matplotlib.pyplot as plt
import seaborn as sns

cm = confusion_matrix(Y_test, Y_test_pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Random Forest Confusion Matrix (Test Set)')
plt.tight_layout()
plt.show()
print('Confusion matrix:')
print(cm)

In [ ]:
# Feature Importance (Mean Decrease in Impurity)
importances = np.zeros(X.shape[1])
for tree, feat_idx in rf_final.trees:
    # Each tree was trained on a subset of features
    # We'll sum the importance for the selected features
    if hasattr(tree, 'root'):
        # Traverse tree to sum infoGain per feature
        def traverse(node, feat_map):
            if node is None or node.value is not None:
                return
            feat_map[feat_idx[node.featureIndex]] += node.infoGain if node.infoGain is not None else 0
            traverse(node.left, feat_map)
            traverse(node.right, feat_map)
        traverse(tree.root, importances)

# Normalize importances
importances = importances / importances.sum()

# Plot
indices = np.argsort(importances)[::-1]
plt.figure(figsize=(10,6))
plt.title('Random Forest Feature Importances (Custom)')
plt.bar(range(X.shape[1]), importances[indices], align='center')
plt.xticks(range(X.shape[1]), [feature_names[i] for i in indices], rotation=90)
plt.tight_layout()
plt.show()

print('Top 10 features:')
for i in indices[:10]:
    print(f'{feature_names[i]}: {importances[i]:.4f}')

In [ ]:
# Tree Complexity Analysis (Average Depth, Node Count)
total_nodes = 0
total_leaves = 0
total_depth = 0
for tree, feat_idx in rf_final.trees:
    def count_nodes(node):
        if node is None:
            return 0
        return 1 + count_nodes(node.left) + count_nodes(node.right)
    def count_leaves(node):
        if node is None:
            return 0
        if node.value is not None:
            return 1
        return count_leaves(node.left) + count_leaves(node.right)
    def get_depth(node):
        if node is None or node.value is not None:
            return 0
        return 1 + max(get_depth(node.left), get_depth(node.right))
    total_nodes += count_nodes(tree.root)
    total_leaves += count_leaves(tree.root)
    total_depth += get_depth(tree.root)

n_trees = len(rf_final.trees)
avg_nodes = total_nodes / n_trees
avg_leaves = total_leaves / n_trees
avg_depth = total_depth / n_trees

print(f"Random Forest (Custom) - Average per tree:")
print(f"  - Nodes: {avg_nodes:.1f}")
print(f"  - Leaves: {avg_leaves:.1f}")
print(f"  - Depth: {avg_depth:.1f}")

## Comparison with Single Decision Tree

Compare the test set performance, bias, and variance of your best single Decision Tree (from Part C) and your custom Random Forest (from Part D). Discuss the effect of ensembling on bias and variance.

In [ ]:
# (Optional) If you have test results from your best Decision Tree, enter them here for comparison
# Example: dt_test_acc = 0.93
# dt_test_precision = ...
# dt_test_recall = ...
# dt_test_f1 = ...

# Print comparison table
print("Comparison of Single Decision Tree vs Random Forest (Test Set):")
print("-"*60)
print(f"{'Model':<20}{'Accuracy':<12}{'Precision':<12}{'Recall':<12}{'F1':<12}")
print("-"*60)
# Uncomment and fill in your Decision Tree results:
# print(f"{'Decision Tree':<20}{dt_test_acc:<12.4f}{dt_test_precision:<12.4f}{dt_test_recall:<12.4f}{dt_test_f1:<12.4f}")
print(f"{'Random Forest':<20}{accuracy:<12.4f}{precision.mean():<12.4f}{recall.mean():<12.4f}{f1.mean():<12.4f}")

print("\nDiscussion:")
print("- Random Forest typically reduces variance (less overfitting) and may slightly increase bias compared to a single tree.")
print("- In most cases, ensemble methods like Random Forest achieve higher test accuracy and more stable predictions.")